In [38]:
import pandas as pd

df = pd.read_csv(r'afteretl.csv')

In [39]:
df.head()

,Unnamed: 0,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,...,drug substances,annual earnings,value_divided,Time,Month,Day,Year,Quarter,Day of Week,Vict Age bucket
0,0,10304468,2020-01-08,01/08/2020,2230,3,Southwest,377,2,624,...,Alcohol,118908,22.3,22,1,8,2020,1,Wednesday,35-60
1,1000,200105946,2020-01-29,01/29/2020,720,1,Central,157,2,624,...,Prescription opioids,27044,7.2,7,1,29,2020,1,Wednesday,35-60
2,2000,200205282,2020-01-27,01/27/2020,810,2,Rampart,249,2,956,...,Heroin,127062,8.1,8,1,27,2020,1,Monday,35-60
3,3000,200110265,2020-04-09,03/31/2020,1930,1,Central,163,1,350,...,Cannabis (Marijuana),144944,19.3,19,3,31,2020,1,Thursday,35-60
4,4000,200404736,2020-01-20,01/19/2020,2030,4,Hollenbeck,437,2,627,...,Alcohol,101813,20.3,20,1,19,2020,1,Monday,0-18


In [44]:
'''# Tworzenie zestawienia wielowymiarowego danych (kostki OLAP)
df = df.groupby( "Year","Quarter",'Month','Day', 'AREA NAME','gender of criminal',  'age of criminal', ).agg(count('*').alias('IncidentCount'))

# Wyświetlenie pierwszych kilku wierszy kostki OLAP
df.show()
'''
# Grupowanie danych i agregacja
df['count'] = 1
df = df.groupby(
    ["Year", "Quarter", "Month", "Day", "AREA NAME", "gender of criminal", "age of criminal"]
).agg({'count': 'sum'}).reset_index()

# Zmieniamy nazwę kolumny 'count' na 'IncidentCount'
df.rename(columns={'count': 'IncidentCount'}, inplace=True)

# Wyświetlenie pierwszych kilku wierszy kostki OLAP
df.head()

,Year,Quarter,Month,Day,AREA NAME,gender of criminal,age of criminal,IncidentCount
0,2020,1,1,2,Southeast,Male,35-60,2
1,2020,1,1,3,Topanga,Female,18-35,2
2,2020,1,1,3,Van Nuys,X-gender,18-35,2
3,2020,1,1,4,Topanga,Male,35-60,2
4,2020,1,1,5,Northeast,Male,18-35,2


In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

# Tworzenie ramki danych


# Grupowanie i agregacja danych
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df_grouped = df.groupby('Date')['IncidentCount'].sum().reset_index()

# Przygotowanie danych do modelu XGBoost
df_grouped['Month'] = df_grouped['Date'].dt.month
df_grouped['Day'] = df_grouped['Date'].dt.day

X = df_grouped[['Month', 'Day']]
y = df_grouped['IncidentCount']

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dopasowanie modelu XGBoost
model = xgb.XGBRegressor(objective ='reg:squarederror', seed=42)
model.fit(X_train, y_train)

# Prognoza na następny rok
next_year_dates = pd.date_range(start=df_grouped['Date'].max() + pd.DateOffset(days=1), end=df_grouped['Date'].max() + pd.DateOffset(years=1), freq='D')
next_year_X = pd.DataFrame({'Month': next_year_dates.month, 'Day': next_year_dates.day})
forecast = model.predict(next_year_X)

# Wykres szeregu czasowego z prognozą
fig = go.Figure()

# Dodanie rzeczywistych danych
fig.add_trace(go.Scatter(x=df_grouped['Date'], y=df_grouped['IncidentCount'], mode='lines+markers', name='Actual'))

# Dodanie prognoz
fig.add_trace(go.Scatter(x=next_year_dates, y=forecast, mode='lines+markers', name='Forecast', line=dict(color='red', dash='dash')))

# Aktualizacja układu wykresu
fig.update_layout(title='Liczba incydentów kryminalnych w czasie z prognozą na następny rok', xaxis_title='Data', yaxis_title='Liczba incydentów')

# Wyświetlenie wykresu
fig.show()
# Zapisywanie wykresu do pliku HTML
fig.write_html("forecast_plot.html")
